### sklearn.ensemble.VotingClassifier
### sklearn.ensemble.VotingRegressor

#### 주요 Hyperparameter
- 분류 : hard(범주형), soft(확률형)

##### VotingClassifier(estimators, *, voting='hard', n_jobs=None, weights=None, flatten_transform=True)
##### VotingRegressor(estimators, *, n_jobs=None, weights=None, verbose=False)

# 분석 코드 - Classification
### Hard Learner

In [1]:
# 라이브러리 및 데이터 로드
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingClassifier, VotingRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, accuracy_score

df = pd.read_csv('../input/big-data-certification-study/breast-cancer-wisconsin.csv', encoding='utf-8')
df.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
# 데이터 분리
X=df.drop(columns=['code','Class'])
y=df[['Class']]
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42)

In [3]:
# 정규화
scaler=MinMaxScaler()
scaler.fit(X_train)
mm_X_train=scaler.transform(X_train)
mm_X_test=scaler.transform(X_test)

In [4]:
# 모델 적용
logit_model=LogisticRegression(random_state=42)
rf_model=RandomForestClassifier(random_state=42)
svm=SVC(random_state=42)

voting_hard=VotingClassifier(estimators=[('lr',logit_model),
                                          ('rf',rf_model),
                                          ('svc',svm)],
                            voting='hard')
voting_hard.fit(mm_X_train, y_train)

for al in (logit_model, rf_model, svm, voting_hard):
    al.fit(mm_X_train, y_train)
    y_pred=al.predict(mm_X_test)
    print(al.__class__.__name__, accuracy_score(y_test,y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544
VotingClassifier 0.9649122807017544


In [5]:
# 혼동행렬, 분류예측 보고서
for al in (logit_model, rf_model, svm, voting_hard):
    y_pred=al.predict(mm_X_test)
    cm_train=confusion_matrix(y_test,y_pred)
    cfr_train=classification_report(y_test,y_pred)
    print(al.__class__.__name__,'혼동행렬 :\n',cm_train,'\n',
          al.__class__.__name__,'분류예측 보고서 :\n',cfr_train,'\n\n')

LogisticRegression 혼동행렬 :
 [[106   5]
 [  2  58]] 
 LogisticRegression 분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171
 


RandomForestClassifier 혼동행렬 :
 [[106   5]
 [  1  59]] 
 RandomForestClassifier 분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.99      0.95      0.97       111
           1       0.92      0.98      0.95        60

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171
 


SVC 혼동행렬 :
 [[106   5]
 [  1  59]] 
 SVC 분류예측 보고서 :
               precision    recall  f1-score   support

           0       0.99      0.95      0.97       111
    

# 분석 코드 - Regression

In [6]:
df2=pd.read_csv('../input/big-data-certification-study/house_price.csv', encoding='utf-8')
df2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [7]:
# 데이터 분리
X=df2.drop(columns=['house_value'])
y=df2[['house_value']]
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [8]:
# 정규화
scale=MinMaxScaler()
scale.fit(X_train)
ms_X_train=scale.transform(X_train)
ms_x_test=scale.transform(X_test)

In [9]:
# 모델 적용
lr_model=LinearRegression()
rfr_model=RandomForestRegressor(random_state=42)

voting_regressor=VotingRegressor(estimators=[('lr',lr_model),('rf',rfr_model)])
voting_regressor.fit(ms_X_train, y_train)
pred_x=voting_regressor.predict(ms_X_train)
voting_regressor.score(ms_X_train, y_train)

0.8128809262725605

In [10]:
pred_y=voting_regressor.predict(ms_x_test)
voting_regressor.score(ms_x_test,y_test)

0.6269061036317223

In [11]:
# RMSE
rmse_train=np.sqrt(mean_squared_error(y_train,pred_x))
rmse_test=np.sqrt(mean_squared_error(y_test,pred_y))
print('Train RMSE :', round(rmse_train),
      '\nTest RMSE :', round(rmse_test))

Train RMSE : 41287 
Test RMSE : 58394
